<a href="https://colab.research.google.com/github/PritishRane/Deep-Learning/blob/main/CNN_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from warnings import filterwarnings
filterwarnings ('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile
import os


In [4]:


# Path to the uploaded zip file in your Google Drive
zip_path = '/content/drive/MyDrive/covid19.zip'
extract_path = '/content/drive/MyDrive/covid19/Covid19-dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)


Dataset extracted to: /content/drive/MyDrive/covid19/Covid19-dataset


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
base_dir = "/content/drive/MyDrive/covid19/Covid19-dataset/Covid19-dataset"
train_dir = os.path.join(base_dir, 'train')

test_dir = os.path.join(base_dir, 'test')

In [7]:
img_width, img_height = 64, 64
batch_size = 32

In [8]:
#Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [9]:


test_datagen = ImageDataGenerator(rescale=1./255)


In [10]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)




test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=1,  # For predictions
    class_mode='categorical',
    shuffle=False
)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [11]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

In [12]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,

)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 692ms/step - accuracy: 0.3840 - loss: 1.1009 - val_accuracy: 0.3939 - val_loss: 1.0732
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.3750 - loss: 1.1628 - val_accuracy: 0.6212 - val_loss: 1.0665
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 145ms/step - accuracy: 0.5045 - loss: 1.0646 - val_accuracy: 0.4848 - val_loss: 1.0132
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.4062 - loss: 1.0786 - val_accuracy: 0.7273 - val_loss: 0.9927
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 141ms/step - accuracy: 0.5890 - loss: 0.9993 - val_accuracy: 0.8182 - val_loss: 0.7377
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step - accuracy: 0.6250 - loss: 0.8907 - val_accuracy: 0.6970 - val_loss: 0.7329
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 143ms/step - accuracy: 0.6360 - loss: 0.8515 - val_accuracy: 0.6818 - val_loss: 0.5752
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.5938 - loss: 0.9579 - val_accuracy: 0.6818 - val_loss

In [14]:
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9481 - loss: 0.3977
Test Loss: 0.515701949596405, Test Accuracy: 0.9242424368858337


In [15]:
predictions = np.argmax(model.predict(test_generator, steps=test_generator.samples), axis=-1)
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


In [16]:
print("Classification Report:\n")
print(classification_report(true_labels, predictions, target_names=class_labels))


Classification Report:

                 precision    recall  f1-score   support

          Covid       1.00      0.96      0.98        26
         Normal       0.90      0.90      0.90        20
Viral Pneumonia       0.86      0.90      0.88        20

       accuracy                           0.92        66
      macro avg       0.92      0.92      0.92        66
   weighted avg       0.93      0.92      0.93        66



In [65]:
model.save("model.h5")

**# LeNet-5**

In [17]:
LeNet5= Sequential([
    Conv2D(6, (5, 5), activation='tanh', input_shape=(img_width, img_height, 3), padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    Conv2D(16, (5, 5), activation='tanh', padding='valid'),
    MaxPooling2D(pool_size=(2, 2), strides=2),

    Flatten(),
    Dense(120, activation='tanh'),
    Dense(84, activation='tanh'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

In [18]:
LeNet5.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
history1 = model.fit( train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 150ms/step - accuracy: 0.6690 - loss: 0.7320 - val_accuracy: 0.8939 - val_loss: 0.4852
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.8438 - loss: 0.5349 - val_accuracy: 0.8333 - val_loss: 0.5209
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 439ms/step - accuracy: 0.7075 - loss: 0.6797 - val_accuracy: 0.7273 - val_loss: 0.4654
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.7188 - loss: 0.6293 - val_accuracy: 0.8485 - val_loss: 0.6057
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 138ms/step - accuracy: 0.7292 - loss: 0.7342 - val_accuracy: 0.7424 - val_loss: 0.4238
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - accuracy: 0.6250 - loss: 0.7721 - val_accuracy: 0.8636 - val_loss: 0.4196
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 219ms/step - accuracy: 0.7454 - loss: 0.6035 - val_accuracy: 0.8485 - val_loss: 0.4319
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.7037 - loss: 0.5719 - val_accuracy: 0.7727 - val_los

In [32]:
loss, accuracy = LeNet5.evaluate(test_generator, steps=test_generator.samples)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5698 - loss: 0.9751
Test Loss: 1.1174923181533813, Test Accuracy: 0.3484848439693451


In [33]:
predictions = np.argmax(LeNet5.predict(test_generator, steps=test_generator.samples), axis=-1)
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step


In [34]:
print("Classification Report:\n")
print(classification_report(true_labels, predictions, target_names=class_labels))


Classification Report:

                 precision    recall  f1-score   support

          Covid       0.42      0.62      0.50        26
         Normal       0.25      0.35      0.29        20
Viral Pneumonia       0.00      0.00      0.00        20

       accuracy                           0.35        66
      macro avg       0.22      0.32      0.26        66
   weighted avg       0.24      0.35      0.29        66



In [66]:
model.save("LetNet5.h5")

# **AlexNet**

In [25]:
from tensorflow.keras.layers import BatchNormalization

In [41]:
AlexNet = Sequential([
    Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=(img_width, img_height, 3), padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),

    Conv2D(256, (5, 5), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),

    Conv2D(384, (3, 3), padding='same', activation='relu'),
    Conv2D(384, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),

    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

In [42]:
AlexNet.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
history2 = model.fit( train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 216ms/step - accuracy: 0.8594 - loss: 0.3969 - val_accuracy: 0.9394 - val_loss: 0.2635
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 191ms/step - accuracy: 0.8125 - loss: 0.4446 - val_accuracy: 0.9242 - val_loss: 0.2762
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 140ms/step - accuracy: 0.7666 - loss: 0.5392 - val_accuracy: 0.8788 - val_loss: 0.2974
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - accuracy: 0.8750 - loss: 0.4119 - val_accuracy: 0.9091 - val_loss: 0.2786
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 142ms/step - accuracy: 0.7686 - loss: 0.5348 - val_accuracy: 0.8788 - val_loss: 0.3063
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.7188 - loss: 0.6767 - val_accuracy: 0.8636 - val_loss: 0.2968
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 180ms/step - accuracy: 0.8022 - loss: 0.4587 - val_accuracy: 0.8636 - val_loss: 0.4192
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 214ms/step - accuracy: 0.6875 - loss: 0.6072 - val_accuracy: 0.8788 - val_loss:

In [44]:
loss, accuracy = AlexNet.evaluate(test_generator, steps=test_generator.samples)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3416 - loss: 1.0975
Test Loss: 1.0982357263565063, Test Accuracy: 0.3636363744735718


In [45]:
predictions = np.argmax(AlexNet.predict(test_generator, steps=test_generator.samples), axis=-1)
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step


In [46]:
print("Classification Report:\n")
print(classification_report(true_labels, predictions, target_names=class_labels))

Classification Report:

                 precision    recall  f1-score   support

          Covid       0.44      0.31      0.36        26
         Normal       0.33      0.80      0.47        20
Viral Pneumonia       0.00      0.00      0.00        20

       accuracy                           0.36        66
      macro avg       0.26      0.37      0.28        66
   weighted avg       0.28      0.36      0.29        66



In [67]:
model.save("AlexNet.h5")

# **GoogleNet**

In [49]:
from tensorflow.keras.layers import AveragePooling2D, concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [50]:
def inception_module(x, filters):
    # 1x1 convolution
    conv1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    # 3x3 convolution
    conv3 = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    conv3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3)

    # 5x5 convolution
    conv5 = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    conv5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5)

    # Max pooling
    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(pool)

    # Concatenate all
    output = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return output

input_layer = Input(shape=(img_width, img_height, 3))

In [52]:
# Initial layers
x = Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(input_layer)
x = MaxPooling2D((3, 3), strides=2, padding='same')(x)
x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D((3, 3), strides=2, padding='same')(x)

# Inception modules
x = inception_module(x, [64, 96, 128, 16, 32, 32])
x = inception_module(x, [128, 128, 192, 32, 96, 64])
x = MaxPooling2D((3, 3), strides=2, padding='same')(x)

x = inception_module(x, [192, 96, 208, 16, 48, 64])
x = inception_module(x, [160, 112, 224, 24, 64, 64])
x = inception_module(x, [128, 128, 256, 24, 64, 64])
x = inception_module(x, [112, 144, 288, 32, 64, 64])
x = inception_module(x, [256, 160, 320, 32, 128, 128])
x = MaxPooling2D((3, 3), strides=2, padding='same')(x)

x = inception_module(x, [256, 160, 320, 32, 128, 128])
x = inception_module(x, [384, 192, 384, 48, 128, 128])

# Final layers
x = AveragePooling2D((2, 2), strides=1, padding='valid')(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
output_layer = Dense(len(train_generator.class_indices), activation='softmax')(x)

In [53]:
googlenet= Model(inputs=input_layer, outputs=output_layer)

In [54]:
googlenet.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [55]:
history3= googlenet.fit( train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 772ms/step - accuracy: 0.4381 - loss: 1.2120 - val_accuracy: 0.3939 - val_loss: 1.0969
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 22s 148ms/step - accuracy: 0.4815 - loss: 1.0942 - val_accuracy: 0.3939 - val_loss: 1.0962
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 178ms/step - accuracy: 0.4100 - loss: 1.0941 - val_accuracy: 0.3939 - val_loss: 1.0914
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 0.5938 - loss: 1.0528 - val_accuracy: 0.3939 - val_loss: 1.0918
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 245ms/step - accuracy: 0.4540 - loss: 1.0700 - val_accuracy: 0.3939 - val_loss: 1.0933
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - accuracy: 0.4444 - loss: 1.0716 - val_accuracy: 0.3939 - val_loss: 1.0918
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 280ms/step - accuracy: 0.4502 - loss: 1.0742 - val_accuracy: 0.3939 - val_loss: 1.0905
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.4375 - loss: 1.0774 - val_accuracy: 0.3939 - val_l

In [56]:
loss, accuracy = googlenet.evaluate(test_generator, steps=test_generator.samples)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7510 - loss: 0.8810
Test Loss: 1.0756728649139404, Test Accuracy: 0.39393940567970276


In [57]:
predictions = np.argmax(googlenet.predict(test_generator, steps=test_generator.samples), axis=-1)
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step


In [58]:
print("Classification Report:\n")
print(classification_report(true_labels, predictions, target_names=class_labels))

Classification Report:

                 precision    recall  f1-score   support

          Covid       0.39      1.00      0.57        26
         Normal       0.00      0.00      0.00        20
Viral Pneumonia       0.00      0.00      0.00        20

       accuracy                           0.39        66
      macro avg       0.13      0.33      0.19        66
   weighted avg       0.16      0.39      0.22        66



In [68]:
model.save("googlenet.h5")

Compareing between model